In [1]:
###### phase 9: apply manual updates from annotathon (20 June 2019) ######

In [2]:
from os import path, makedirs
import glob
import json
import re

In [3]:
fixed = {}
todo_list = {}

def solve_issue(bgc_id, issue):
    if bgc_id not in fixed:
        fixed[bgc_id] = []
    fixed[bgc_id].append(issue)
    
def add_todo(bgc_id, issue, indexes):
    if bgc_id not in todo_list:
        todo_list[bgc_id] = {}
    todo_list[bgc_id][issue] = indexes

In [4]:
def pre_annotathon_fix(data):
    bgc_id = data["cluster"]["mibig_accession"]
    
    if False:
        pass
    elif bgc_id == "BGC0000247":
        assert data["cluster"]["saccharide"]["glycosyltransferases"][0]["specificity"] == "D-olivose and D-mycarose"
        assert len(data["cluster"]["saccharide"]["glycosyltransferases"]) == 4
        new_gts = []
        new_gts.append(data["cluster"]["saccharide"]["glycosyltransferases"][0].copy())
        new_gts[-1]["specificity"] = "D-olivose"
        new_gts.append(data["cluster"]["saccharide"]["glycosyltransferases"][0].copy())
        new_gts[-1]["specificity"] = "D-mycarose"
        new_gts.extend(data["cluster"]["saccharide"]["glycosyltransferases"][1:])
        data["cluster"]["saccharide"]["glycosyltransferases"] = new_gts
    elif bgc_id == "BGC0000356":
        # a_substr_spec.proteinogenic
        assert data["cluster"]["nrp"]["nrps_genes"][0]["modules"][0]["a_substr_spec"]["proteinogenic"] == ["Tryptophan / Proline"]
        data["cluster"]["nrp"]["nrps_genes"][0]["modules"][0]["a_substr_spec"]["proteinogenic"] = ["Tryptophan", "Proline"]
        assert data["cluster"]["nrp"]["nrps_genes"][0]["modules"][1]["a_substr_spec"]["proteinogenic"] == ["Proline / Tryptophan"]
        data["cluster"]["nrp"]["nrps_genes"][0]["modules"][1]["a_substr_spec"]["proteinogenic"] = ["Tryptophan", "Proline"]
        solve_issue(bgc_id, "a_substr_spec.proteinogenic")
        pass
    elif bgc_id == "BGC0000424":
        # a_substr_spec.proteinogenic
        assert data["cluster"]["nrp"]["nrps_genes"][0]["modules"][0]["a_substr_spec"]["proteinogenic"] == ["C10:0; C12:0; C14:0"]
        data["cluster"]["nrp"]["nrps_genes"][0]["modules"] = data["cluster"]["nrp"]["nrps_genes"][0]["modules"][1:]
        solve_issue(bgc_id, "a_substr_spec.proteinogenic")
        pass    
    elif bgc_id == "BGC0000805":
        assert data["cluster"]["saccharide"]["glycosyltransferases"][2]["specificity"] == "chinovosamine AND N-acetyl-D-glucosamine"
        assert len(data["cluster"]["saccharide"]["glycosyltransferases"]) == 5
        new_gts = []
        new_gts.extend(data["cluster"]["saccharide"]["glycosyltransferases"][:2])
        new_gts.append(data["cluster"]["saccharide"]["glycosyltransferases"][2].copy())
        new_gts[-1]["specificity"] = "Chinovosamine"
        new_gts.append(data["cluster"]["saccharide"]["glycosyltransferases"][2].copy())
        new_gts[-1]["specificity"] = "N-acetyl-D-glucosamine"
        new_gts.extend(data["cluster"]["saccharide"]["glycosyltransferases"][3:])
        data["cluster"]["saccharide"]["glycosyltransferases"] = new_gts
    elif bgc_id == "BGC0000959":
        # a_substr_spec.proteinogenic
        assert data["cluster"]["nrp"]["nrps_genes"][0]["modules"][0]["a_substr_spec"]["proteinogenic"] == ["L-tyrosin; L-phenylalanine; L-tryptophan; L-4-fluorophenylalanine "]
        data["cluster"]["nrp"]["nrps_genes"][0]["modules"][0]["a_substr_spec"]["proteinogenic"] = ["Tyrosine", "Phenylalanine", "Tryptophan"]
        assert "nonproteinogenic" not in data["cluster"]["nrp"]["nrps_genes"][0]["modules"][0]["a_substr_spec"]
        data["cluster"]["nrp"]["nrps_genes"][0]["modules"][0]["a_substr_spec"]["nonproteinogenic"] = ["L-4-fluorophenylalanine"]
        solve_issue(bgc_id, "a_substr_spec.proteinogenic")
        pass    
    elif bgc_id == "BGC0001133":
        # a_substr_spec.proteinogenic
        assert data["cluster"]["nrp"]["nrps_genes"][0]["modules"][2]["a_substr_spec"]["proteinogenic"] == ["Tyr","Phe"]
        data["cluster"]["nrp"]["nrps_genes"][0]["modules"][2]["a_substr_spec"]["proteinogenic"] = ["Tyrosine", "Phenylalanine"]
        solve_issue(bgc_id, "a_substr_spec.proteinogenic")
        pass
    
    return

In [5]:
def capitalize(text): # capitalize the very first letter of the text
    first_letter_index = re.search("[a-zA-Z]{1}", text).start()
    return text[:first_letter_index] + text[first_letter_index].upper() + text[first_letter_index + 1:]

def search_and_replace_value(input_dict, value_from, value_to): # very inefficient, please fix when you have time
    count = 0
    if isinstance(input_dict, dict):
        new_dict = {}
        for key in input_dict:
            if isinstance(input_dict[key], str):                
                if input_dict[key].strip() == value_from:
                    if len(value_to) > 1:
                        # shouldn't happen
                        print(value_to)
                        sys.exit(1)
                    elif value_to[0] == "<Delete>":
                        pass
                    else:
                        new_dict[key] = value_to[0]
                    count += 1
            else:
                res = search_and_replace_value(input_dict[key], value_from, value_to)
                count += res[0]
                new_dict[key] = res[1]
        input_dict = new_dict
    elif isinstance(input_dict, list):
        new_list = []
        for i, valu in enumerate(input_dict):
            if isinstance(valu, str):
                if valu.strip() == value_from:
                    if len(value_to) > 0:
                        for valuto in value_to:
                            new_list.append(valuto)
                    elif value_to[0] == "<Delete>":
                        pass
                    else:
                        new_list.append(value_to[0])
                    count += 1
            else:
                res = search_and_replace_value(valu, value_from, value_to)
                count += res[0]
                new_list.append(res[1])
        input_dict = new_list
    elif isinstance(input_dict, str):
        sys.exit(1) # shouldn't happen with our data
    return count, input_dict

enum_fixes = {}
with open("../../inputs/from_annotathon_20june2019/enums.tsv", "r") as tsv:
    for line in tsv:
        cols = line.rstrip("\n").split("\t")
        if len(cols[0]) > 0:
            cur_property = [prop for prop in cols[0].split(".") if prop not in ["properties", "items", "enum"]]
            # cur_property holds the old schema, we can't really use it now
        old_val = cols[1] # it is trimmed left and right
        new_val = [cols[2]]
        if len(new_val[0]) < 1:
            new_val = [capitalize(valu) for valu in cols[3].split(";")]
        assert len(new_val) > 0
        bgc_ids = cols[4].split(",")
        for bgc_id in bgc_ids:
            if bgc_id not in enum_fixes:
                enum_fixes[bgc_id] = {}
            enum_fixes[bgc_id][old_val] = new_val

proteinogenic_aa = [
    "Methionine",
    "Glutamate",
    "Glycine",
    "Valine",
    "Phenylalanine",
    "Arginine",
    "Cysteine",
    "Glutamine",
    "Leucine",
    "Asparagine",
    "Isoleucine",
    "Aspartate",
    "Tryptophan",
    "Threonine",
    "Alanine",
    "Tyrosine",
    "Histidine",
    "Proline",
    "Lysine",
    "Serine"
]
            
def enum_fix(data):
    bgc_id = data["cluster"]["mibig_accession"]
    fix_count = 0
    for old_val in enum_fixes.get(bgc_id, {}):
        new_val = enum_fixes[bgc_id][old_val]
        if old_val in [
                "D-olivose and D-mycarose",
                "chinovosamine AND N-acetyl-D-glucosamine"
            ]:
            fix_count += 1 # fixed in pre-annotathon step
        else:
            res = search_and_replace_value(data, old_val, new_val)
            if res[0] < 1:
                print("Can't find fixable enum {}".format(bgc_id))
                sys.exit(1)
            else:
                fix_count += 1
    if fix_count > 0:
        # post-process for nrps nonproteinogenic substrate specificities
        if "nrp" in data["cluster"]:
            for gene in data["cluster"]["nrp"].get("nrps_genes", []):
                for module in gene.get("modules", []):
                    if "a_substr_spec" in module:
                        a_spec = module["a_substr_spec"]
                        proteinogenic = set()
                        non_proteinogenic = set()
                        for val in a_spec.get("proteinogenic", []):
                            assert (val in proteinogenic_aa)
                            proteinogenic.add(val)
                        for val in a_spec.get("nonproteinogenic", []):
                            if val.startswith("Proteinogenic:"):
                                val = capitalize(val.split("Proteinogenic:")[-1])
                                assert (val in proteinogenic_aa)
                                proteinogenic.add(val)
                            elif val == "<Delete>":
                                sys.exit(1)
                            else:
                                non_proteinogenic.add(val)
                        if len(proteinogenic) > 0:
                            a_spec["proteinogenic"] = list(proteinogenic)
                        else:
                            a_spec.pop("proteinogenic", None)
                        if len(non_proteinogenic) > 0:
                            a_spec["nonproteinogenic"] = list(non_proteinogenic)
                        else:
                            a_spec.pop("nonproteinogenic", None)
                        
        solve_issue(bgc_id, "enum")
        
    return

In [7]:
def multi_loci_fix(data):
    return

In [8]:
def gene_annots_fix(data):
    return

In [9]:
def taxonomy_fix(data):
    return

In [10]:
def update(data):
    pre_annotathon_fix(data)
    enum_fix(data)
    multi_loci_fix(data)
    gene_annots_fix(data)
    taxonomy_fix(data)
    return

In [11]:
input_path = "../../preprocessed/p8-json/"
output_folder = "../../preprocessed/p9-json/"

if not path.exists(output_folder):
    makedirs(output_folder)

for json_path in sorted(glob.glob(path.join(input_path, "BGC*.json"))):
    with open(json_path, "r") as json_file:
        bgc_id = path.basename(json_path).split(".")[0]
        data = json.load(json_file)
        print("Scanning {}".format(bgc_id))
        update(data)
        with open(path.join(output_folder, "{}.json".format(bgc_id)), "w") as o:
            o.write(json.dumps(data, indent=4, separators=(',', ': '), sort_keys=True))
            
print("All data fetched!")

Scanning BGC0000001
Scanning BGC0000002
Scanning BGC0000003
Scanning BGC0000004
Scanning BGC0000005
Scanning BGC0000006
Scanning BGC0000007
Scanning BGC0000008
Scanning BGC0000009
Scanning BGC0000010
Scanning BGC0000011
Scanning BGC0000012
Scanning BGC0000013
Scanning BGC0000014
Scanning BGC0000015
Scanning BGC0000016
Scanning BGC0000017
Scanning BGC0000018
Scanning BGC0000019
Scanning BGC0000020
Scanning BGC0000021
Scanning BGC0000022
Scanning BGC0000023
Scanning BGC0000024
Scanning BGC0000025
Scanning BGC0000026
Scanning BGC0000027
Scanning BGC0000028
Scanning BGC0000029
Scanning BGC0000030
Scanning BGC0000031
Scanning BGC0000032
Scanning BGC0000033
Scanning BGC0000034
Scanning BGC0000035
Scanning BGC0000036
Scanning BGC0000037
Scanning BGC0000038
Scanning BGC0000039
Scanning BGC0000040
Scanning BGC0000041
Scanning BGC0000042
Scanning BGC0000043
Scanning BGC0000044
Scanning BGC0000045
Scanning BGC0000046
Scanning BGC0000047
Scanning BGC0000048
Scanning BGC0000049
Scanning BGC0000050


Scanning BGC0000437
Scanning BGC0000438
Scanning BGC0000439
Scanning BGC0000440
Scanning BGC0000441
Scanning BGC0000442
Scanning BGC0000443
Scanning BGC0000444
Scanning BGC0000445
Scanning BGC0000446
Scanning BGC0000447
Scanning BGC0000448
Scanning BGC0000449
Scanning BGC0000450
Scanning BGC0000451
Scanning BGC0000452
Scanning BGC0000453
Scanning BGC0000454
Scanning BGC0000455
Scanning BGC0000456
Scanning BGC0000457
Scanning BGC0000458
Scanning BGC0000459
Scanning BGC0000460
Scanning BGC0000461
Scanning BGC0000462
Scanning BGC0000463
Scanning BGC0000464
Scanning BGC0000465
Scanning BGC0000466
Scanning BGC0000467
Scanning BGC0000468
Scanning BGC0000469
Scanning BGC0000470
Scanning BGC0000471
Scanning BGC0000472
Scanning BGC0000473
Scanning BGC0000474
Scanning BGC0000475
Scanning BGC0000476
Scanning BGC0000477
Scanning BGC0000478
Scanning BGC0000479
Scanning BGC0000480
Scanning BGC0000481
Scanning BGC0000482
Scanning BGC0000483
Scanning BGC0000484
Scanning BGC0000485
Scanning BGC0000486


Scanning BGC0000963
Scanning BGC0000964
Scanning BGC0000965
Scanning BGC0000966
Scanning BGC0000967
Scanning BGC0000968
Scanning BGC0000969
Scanning BGC0000970
Scanning BGC0000971
Scanning BGC0000972
Scanning BGC0000973
Scanning BGC0000974
Scanning BGC0000975
Scanning BGC0000976
Scanning BGC0000977
Scanning BGC0000978
Scanning BGC0000979
Scanning BGC0000980
Scanning BGC0000981
Scanning BGC0000982
Scanning BGC0000983
Scanning BGC0000984
Scanning BGC0000985
Scanning BGC0000986
Scanning BGC0000988
Scanning BGC0000989
Scanning BGC0000990
Scanning BGC0000991
Scanning BGC0000992
Scanning BGC0000993
Scanning BGC0000994
Scanning BGC0000995
Scanning BGC0000996
Scanning BGC0000997
Scanning BGC0000998
Scanning BGC0000999
Scanning BGC0001000
Scanning BGC0001001
Scanning BGC0001002
Scanning BGC0001003
Scanning BGC0001004
Scanning BGC0001005
Scanning BGC0001006
Scanning BGC0001007
Scanning BGC0001008
Scanning BGC0001009
Scanning BGC0001010
Scanning BGC0001011
Scanning BGC0001012
Scanning BGC0001013


Scanning BGC0001550
Scanning BGC0001551
Scanning BGC0001552
Scanning BGC0001553
Scanning BGC0001554
Scanning BGC0001555
Scanning BGC0001556
Scanning BGC0001557
Scanning BGC0001558
Scanning BGC0001559
Scanning BGC0001560
Scanning BGC0001561
Scanning BGC0001562
Scanning BGC0001563
Scanning BGC0001564
Scanning BGC0001565
Scanning BGC0001566
Scanning BGC0001567
Scanning BGC0001568
Scanning BGC0001569
Scanning BGC0001570
Scanning BGC0001571
Scanning BGC0001572
Scanning BGC0001573
Scanning BGC0001574
Scanning BGC0001575
Scanning BGC0001576
Scanning BGC0001577
Scanning BGC0001578
Scanning BGC0001579
Scanning BGC0001580
Scanning BGC0001581
Scanning BGC0001582
Scanning BGC0001583
Scanning BGC0001584
Scanning BGC0001585
Scanning BGC0001586
Scanning BGC0001587
Scanning BGC0001588
Scanning BGC0001589
Scanning BGC0001590
Scanning BGC0001591
Scanning BGC0001592
Scanning BGC0001593
Scanning BGC0001594
Scanning BGC0001595
Scanning BGC0001596
Scanning BGC0001597
Scanning BGC0001598
Scanning BGC0001599


In [12]:
### update fixed list ###
with open("../../preprocessed/reports/p9-fixed_list.tsv", "w") as o:
    for bgc_id in fixed:
        o.write("{}\t{}\n".format(bgc_id, ";".join(fixed[bgc_id])))

In [13]:
### update todo list ###
with open("../../preprocessed/reports/p9-todo_list.tsv", "w") as o:
    for bgc_id in todo_list:
        for issue in todo_list[bgc_id]:
            o.write("{}\t{}\t{}\n".format(bgc_id, issue, ";".join([str(idx) for idx in todo_list[bgc_id][issue]])))